# HSMA Introduction to Prophet
### Code along lecture 2: Creating a basic Prophet model

**In this code along lecture you will learn:**
* How to fit a `Prophet` model to a time series
* How to obtain a point forecast and prediction intervals using `Prophet`
* How to plot the model forecasts and components
---

### Library imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Prophet Imports

In [ ]:
import prophet

from prophet import Prophet # Prophet class for forecasters
from prophet.plot import plot_plotly, plot_components_plotly # create interactive charts

# should be version 1.0 or above
prophet.__version__

### Utility functions

In [ ]:
def prophet_training_data(y_train):
    '''
    Converts a standard pandas datetimeindexed dataframe
    for time series into one suitable for Prophet
    Parameters:
    ---------
    y_train: pd.DataFrame
        univariate time series data

    Returns:
    --------
        pd.DataFrame in Prophet format
        columns = ['ds', 'y']
    '''
    prophet_train = pd.DataFrame(y_train.index)
    prophet_train['y'] = y_train.to_numpy()
    prophet_train.columns = ['ds', 'y']

    return prophet_train

### **Step 1**: Load and preprocess data

In [ ]:
url = 'https://raw.githubusercontent.com/hsma4/module_9_b/master/data' \
      + '/resp_admits_day.csv'
y_train = pd.read_csv(url,
                      parse_dates=True,
                      dayfirst=True,
                      index_col='date')
y_train.index.freq = 'D'

prophet_train = prophet_training_data(y_train)
prophet_train.tail()


### **Step 2**: Fit a basic Prophet model

Fitting a basic Prophet model is relatively straightforward.  We need to create a `Prophet` object

```python
model = Prophet()
```
Unlike our naive forecasting where we specified the prediction interval width at prediction time, with Prophet we need to specify the interval width as we create the model.  We do this using the parameter `interval_width`.  By default this produces a 80\% prediction interval.  Note we specify a $1-\alpha$ interval width (unlike in our naive models where we specified $\alpha$)

```python
model = Prophet(interval_width=0.95)
```

and call `.fit()` passing in the training data.

```python
model.fit(y_train)
```

> You might find that Prophet warns you that it has disabled **daily seasonality**.  This is slightly confusing terminology.  What it means is that it is not fitting intra-day, e.g. hourly, seasonality.

In [ ]:
# fit a basic prophet model with 0.95 PIs

In [ ]:
help(Prophet)

In [ ]:
model = Prophet(interval_width=0.95, # 80% by default
                daily_seasonality=False) # turning off hourly seasonality (i.e. seasonality within a day)
_ = model.fit(prophet_train)

# **Step 3**: Forecasting

There are two steps to making a forecast with a fitted `Prophet` model.  You must first create a **future dataframe**.  This is a `pd.DataFrame` with a single column 'ds' that runs from the start of the training period until the end of the training period + a horizon.  Prophet makes this easy by including a `make_future_dataframe()` method.

```python
future = model.make_future_dataframe(periods=28)
```

Once we have the future dataframe we can pass that to the predict method.

```python
prophet_forecast = model.predict(future)
```

This returns a `pd.DataFrame` that contains both the in-sample fitted values and the out of sample forecasts. It contains all of the components of the fitted model. For example the last 5 rows of a forecast are:

In [ ]:
# make future data frame
# only ds column
future = model.make_future_dataframe(periods=28)
future.head()

In [ ]:
# compare to training data...
prophet_train.tail(2)

In [ ]:
# predict
# Prophet is a regression model
# we get trend, weekly components, etc
# its additive so adds together to get yhat
prophet_forecast = model.predict(future)
prophet_forecast.head()

In [ ]:
# last 2 predictions
prophet_forecast.tail(2)

### **Step 4**: Plot the fitted values and the forecast

In [ ]:
# real data is black dots
# point forecast is the dark blue line
# light blue is 95% PI
model.plot(prophet_forecast);

### Side bar: using plotly for interactive 

In [ ]:
plot_plotly(model, prophet_forecast)

### **Step 5:** Plotting Prophet's components

In [ ]:
# 1. trend showing 2 change points
# 2. weekly pattern (addtitive model; so detract 10 pts from mean predication on a sunday)
# 3. fourier series by month
ax = model.plot_components(prophet_forecast)

### Plotly side bar...

In [ ]:
plot_components_plotly(model, prophet_forecast)